### IMPORTAÇÕES DAS BIBLIOTECAS E APLICAÇÃO DAS REGRAS

In [1]:
from utils import *
from auth import *
from datetime import datetime
import locale
import numpy as np

In [2]:

#* Definir localização para português
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

#* ID da planilha do BD
spreadsheet_id = '1yAbwVoG3H1F70UTVSqMVLfmAjWEL0f7viqfqZfZAsys'

### CAPTURA DOS DADOS DA PLANILHA

In [3]:

#* Conectando a Sheets
sheet_client                  = authenticate_gsheets(service_account_json=service_account_json, sheet_scopes=sheets_scopes)

print(sheet_client)

#* Pegando as configurações de taxonomia da planilha da mídia correspondente
sheet                         = sheet_client.open_by_key(spreadsheet_id)    

#* Capturando aba Planilha Vendas Mercado Livre - COLE AQUI
wks_ml_vendas                 = sheet.worksheet_by_title(title="Planilha Vendas Mercado Livre - COLE AQUI")
#* Capturando aba Planilha ML - Enviados
wks_ml_enviados               = sheet.worksheet_by_title(title="ML - Enviados")

#* Capturando aba Planilha ML - Cancelados
wks_ml_cancelados             = sheet.worksheet_by_title(title="ML - Cancelados")

#* Capturando aba Planilha ML - Cancelados
wks_ml_reembolso              = sheet.worksheet_by_title(title="ML - Devolução / Reembolso")

#* Gera um dataframe com a planilha de vendas do ML
sheet_df                      = wks_ml_vendas.get_as_df(has_header=True, start="A6", end="AK1000")

A função authenticate_gsheets foi chamada!


### CAPTURA DAS LISTAS DE INFORMAÇÕES NECESSÁRIAS

In [4]:

#* Captura a lista de colunas desejadas
colunas_desejadas = colunas_desejadas_ml()
#* Captura a lista de status cancelados para remover
status_a_remover = status_cancelados_ml()
#* Captura a lista de status de reembolso e devolução
status_devolucao = status_devolucao_ml()

### DATAFRAME CANCELADOS

In [5]:

#! Aplica as regras para cancelados - ML
cancelados_ml = df_cancelados_ml(df=sheet_df, colunas_desejadas=colunas_desejadas, status_cancelado=status_a_remover)

In [6]:

#! Monta dataframe final de cancelados
df_cancelados_final = cria_df_cancelados_ml(cancelados_ml)
df_cancelados_final.head()

,Data,Número da Venda,Item (nome),MarketPlace,Status
37,19/02/2024,'2000005426297255,Biscoito Baunilha Importado Alemanha 400g Memb...,Mercado Livre,Você cancelou a venda
47,18/02/2024,'2000007632428110,"Mesa Redonda Dobrável Casa & Jardim - 83,8 Cm",Mercado Livre,Você cancelou a venda
207,13/02/2024,'2000005406926763,Barril De Cerveja Alemã Hofjäger Weizen Trigo ...,Mercado Livre,Cancelada pelo comprador
223,13/02/2024,'2000005405161691,Óleo De Extra Virgem O-live & Co Vidro Sem Glúten,Mercado Livre,Pacote cancelado pelo Mercado Livre
234,13/02/2024,'2000005404388759,Ventilador De Mesa 40cm 127v Xtreme Confort 9 ...,Mercado Livre,Você cancelou a venda


### DATAFRAME REEMBOLSO

In [7]:

#! Aplica as regras para reembolso - ML
reembolso_ml = df_reembolso_ml(df=sheet_df, colunas_desejadas=colunas_desejadas, status_devolucao_ml=status_devolucao)

In [8]:

#! Monta dataframe final de cancelados
df_reembolso_final = cria_df_reembolso_ml(reembolso_ml)

### DATAFRAME ENVIADOS ML

In [9]:

#! Aplica as regras e formata o dataframe inicial
enviados_ml = df_enviados_ml(df = sheet_df, colunas_desejadas=colunas_desejadas, status_cancelado=status_a_remover, status_devolucao=status_devolucao)

c:\Users\Usuário\Desktop\Scripts\bd_vendas_mercado_livre\utils.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Receita por produtos (BRL)'] = df['Receita por produtos (BRL)'].str.replace(',', '.')
c:\Users\Usuário\Desktop\Scripts\bd_vendas_mercado_livre\utils.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Receita por produtos (BRL)'] = df['Receita por produtos (BRL)'].astype(float)


In [10]:

#! captura o dataframe inicial e aplica as regras gerando o final do mercado livre
df_final_enviados_ml = cria_df_enviados_ml(enviados_ml)
df_final_enviados_ml.head(3)

,Data,Número da Venda,Item (nome),MarketPlace,Status,Valor Nota (Total),Repasse,Imposto,Valor do Imposto
674,01/02/2024,'2000007506994184,Carrinho Modular Ferramenta Profissional 3 Em ...,Mercado Livre,Entregue,1799.90,1495.51,0.00,0.00
645,01/02/2024,'2000007514536134,Carrinho De Carga Plataforma Dobrável - Até 15...,Mercado Livre,Entregue dia 17 de fevereiro,0.00,448.20,0.06,0.00
646,01/02/2024,'2000007514387796,Óleo Extra Virgem De Oliva O-live & Co Vidro S...,Mercado Livre,Entregue,78.99,62.72,0.06,4.74


### CHECAGEM / REMOÇÃO DE DUPLICIDADES

In [11]:

#? Criando dataframes dos dados já existentes na planilha de enviados / cancelados e reembolso / devolução

df_desejado_cancelados = captura_df_desejado(tipo="Cancelados", sheet=sheet)

df_desejado_reembolso  = captura_df_desejado(tipo="Reembolso", sheet=sheet)

df_desejado_enviados   = captura_df_desejado(tipo="Enviados", sheet=sheet)
df_desejado_cancelados

,Data,Número da Venda,Item (Nome),MarketPlace,Status
0,19/02/2024,2000005426297255,Biscoito Baunilha Importado Alemanha 400g Memb...,Mercado Livre,Você cancelou a venda
1,18/02/2024,2000007632428110,"Mesa Redonda Dobrável Casa & Jardim - 83,8 Cm",Mercado Livre,Você cancelou a venda
2,13/02/2024,2000005406926763,Barril De Cerveja Alemã Hofjäger Weizen Trigo ...,Mercado Livre,Cancelada pelo comprador
3,13/02/2024,2000005405161691,Óleo De Extra Virgem O-live & Co Vidro Sem Glúten,Mercado Livre,Pacote cancelado pelo Mercado Livre
4,13/02/2024,2000005404388759,Ventilador De Mesa 40cm 127v Xtreme Confort 9 ...,Mercado Livre,Você cancelou a venda
5,12/02/2024,2000007588436536,Carrinho Cromado Utilitário Multiuso C/ Tampa ...,Mercado Livre,Você cancelou a venda
6,12/02/2024,2000007587593666,Robo Limpador Aspirador De Piscina Zx300 Sobe ...,Mercado Livre,Cancelada pelo comprador
7,10/02/2024,2000007580710376,Chopeira Philco - Barril 5 Litros - Compresso...,Mercado Livre,Venda cancelada. Não envie.
8,07/02/2024,2000005385977471,Ombrelone Central Guarda Sol Com Luz Led Solar...,Mercado Livre,Cancelada pelo comprador
9,07/02/2024,2000007553241140,Purificador De Água C/ Painel Touch Electrolux...,Mercado Livre,Cancelada pelo comprador


In [12]:

#? Checagem de duplicidade para enviados, cancelados e reembolso / devolução

check_duplicidades_cancelados = check_duplicidade(df_cancelados_final, df_desejado_cancelados)

check_duplicidades_reembolso  = check_duplicidade(df_reembolso_final, df_desejado_reembolso)

check_duplicidades_enviados   = check_duplicidade(df_final_enviados_ml, df_desejado_enviados)
check_duplicidades_cancelados

,Data,Número da Venda,Item (nome),MarketPlace,Status


### COLANDO OS VALORES NA PLANILHA

In [15]:

#? Cola os novos valores que não constam na planilha de enviados, cancelados e reembolso/devolução

colar_cancelados = paste_values(sheet_client, spreadsheet_id, wks_ml_cancelados, check_duplicidades_cancelados)

#colar_reembolso = paste_values(sheet_client, spreadsheet_id, wks_ml_reembolso, check_duplicidades_reembolso)

#colar_enviados = paste_values(sheet_client, spreadsheet_id, wks_ml_enviados, check_duplicidades_enviados)